In [1]:
import csv
import pandas as pd
import numpy as np
import jieba
import jieba.analyse
import time
import requests
import json

POS = 'pos'
NEG = 'neg'
labels = []
contents = []
header = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.5060.66 Safari/537.36 Edg/103.0.1264.44"}
flag = NEG;

In [ ]:
with open(f'data_{flag}.csv', 'w', newline='', encoding='utf-8') as f:
    fieldnames = ['label', 'content']
    writer = csv.DictWriter(f, fieldnames=fieldnames)
    writer.writeheader()
    productids = [100028235472,100016864291,100035927374,100016931023,100036448552]
    for productid in productids:
        for page in range(0,20):
            url = f'https://club.jd.com/comment/productPageComments.action?callback=fetchJSON_comment98&productId={productid}&score=1&sortType=5&page={page}&pageSize=10&isShadowSku=0&fold=1'                              # score = 3 is good comment, 1 is bad
            response = requests.get(url, headers=header)
            data = response.text
            jd = json.loads(data.lstrip('fetchJSON_comment98vv12345(').rstrip(');'))
            data_list = jd['comments']
            for data in data_list:
                print(data['content'])
                writer.writerow({'label':f'{flag}','content':data['content']})
            time.sleep(1)
    f.close()

In [23]:
stop_list = pd.read_csv('stop_words.txt',
                        encoding='utf-8',)['t'].to_list()
print(stop_list)

['一个', '一件', '一块', '一天', '一样', '一点', '外观', '屏幕', '手机', '拍照', '效果', '运行', '速度', '一个月', '不到', '不好', '不是', '不能', '不行']


In [24]:
#中文分词函数
def txt_cut(juzi):
    return [w for w in jieba.lcut(juzi) if w not in stop_list]
#写入分词结果
fw = open(f'fenci_data_{flag}.csv', "w", newline = '',encoding = 'utf-8')
writer = csv.writer(fw)
writer.writerow(['label','content'])

# # 使用csv.DictReader读取文件中的信息
labels = []
contents = []
print(contents.__len__())
file = f"data_{flag}.csv"
with open(file, "r", encoding="UTF-8") as f:
    reader = csv.DictReader(f)
    for row in reader:
        # 数据元素获取
        if row['label'] == '0':
            res = 0
        else:
            res = 1
        labels.append(res)
        content = row['content']
        seglist = txt_cut(content)
        output = ' '.join(list(seglist))            #空格拼接
        contents.append(output)

        #文件写入
        tlist = []
        tlist.append(res)
        tlist.append(output)
        writer.writerow(tlist)
f.close()
print(labels[:5])
print(contents[:5])
print(contents.__len__())
fw.close()

0
[1, 1, 1, 1, 1]
['昨天 刚买 的 今天 就 降价 30 块钱', '千元 左右 的 机 就是 这 样子 了', '用 了 快 了 时不时 的 没 信号', '显示 有 赠品 ！ 为什么 收不到', '快递 包裹 简陋 ， 没有 装防 摔 的 气囊 ！']
1000


In [25]:
# -*- coding:utf-8 -*-
import csv
import pandas as pd
import numpy as np
import jieba
import jieba.analyse
from scipy.sparse import coo_matrix
from sklearn import feature_extraction
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

#----------------------------------第一步 读取文件--------------------------------
with open(f'fenci_data_{flag}.csv', 'r', encoding='UTF-8') as f:
    reader = csv.DictReader(f)
    labels = []
    contents = []
    for row in reader:
        labels.append(row['label']) #0-好评 1-差评
        contents.append(row['content'])

print(labels[:5])
print(contents[:5])

['1', '1', '1', '1', '1']
['昨天 刚买 的 今天 就 降价 30 块钱', '千元 左右 的 机 就是 这 样子 了', '用 了 快 了 时不时 的 没 信号', '显示 有 赠品 ！ 为什么 收不到', '快递 包裹 简陋 ， 没有 装防 摔 的 气囊 ！']


In [26]:
#----------------------------------第二步 数据预处理--------------------------------
#将文本中的词语转换为词频矩阵 矩阵元素a[i][j] 表示j词在i类文本下的词频
vectorizer = CountVectorizer()
print(vectorizer.fit_transform(contents))

  (0, 1505)	1
  (0, 610)	1
  (0, 416)	1
  (0, 2394)	1
  (0, 40)	1
  (0, 886)	1
  (1, 673)	1
  (1, 1102)	1
  (1, 1078)	1
  (1, 1610)	1
  (2, 1490)	1
  (2, 482)	1
  (3, 1509)	1
  (3, 2196)	1
  (3, 341)	1
  (3, 1425)	1
  (4, 1216)	1
  (4, 655)	1
  (4, 1945)	1
  (4, 1676)	1
  (4, 2098)	1
  (4, 1652)	1
  (5, 850)	1
  (5, 2197)	1
  (5, 1763)	1
  :	:
  (998, 574)	1
  (998, 713)	1
  (998, 2123)	1
  (998, 1218)	1
  (998, 2150)	1
  (998, 641)	1
  (999, 482)	1
  (999, 1509)	2
  (999, 1216)	2
  (999, 1676)	1
  (999, 1237)	1
  (999, 2310)	1
  (999, 171)	1
  (999, 1807)	1
  (999, 1299)	1
  (999, 1128)	1
  (999, 888)	1
  (999, 1112)	1
  (999, 762)	1
  (999, 1067)	1
  (999, 127)	1
  (999, 209)	2
  (999, 1836)	1
  (999, 389)	1
  (999, 1408)	1


In [27]:
#该类会统计每个词语的tf-idf权值
transformer = TfidfTransformer()
#左起第一个fit_transform()计算tf-idf 第二个fit_transform()将文本转为词频矩阵
tfidf = transformer.fit_transform(vectorizer.fit_transform(contents))
# print(tfidf[0])
for n in tfidf[:5]:
    print(n)
print(type(tfidf))

  (0, 2394)	0.2852320291484702
  (0, 1505)	0.4682689307606282
  (0, 886)	0.45778255951930047
  (0, 610)	0.3594016671551359
  (0, 416)	0.4084172107525733
  (0, 40)	0.4402588441352423
  (0, 1610)	0.5006851631756772
  (0, 1102)	0.5410542142472095
  (0, 1078)	0.33827825718383264
  (0, 673)	0.584929504588159
  (0, 1490)	0.7984201973398706
  (0, 482)	0.6021006464701414
  (0, 2196)	0.4291134360790619
  (0, 1509)	0.45172563801430576
  (0, 1425)	0.6115688099428245
  (0, 341)	0.48763633749148194
  (0, 2098)	0.4842406192099647
  (0, 1945)	0.4570297887523602
  (0, 1676)	0.194047361559403
  (0, 1652)	0.4842406192099647
  (0, 1216)	0.30477806111836686
  (0, 655)	0.43772339729563964
<class 'scipy.sparse.csr.csr_matrix'>


In [28]:
# 获取词袋模型中的所有词语
word = vectorizer.get_feature_names_out()
print("单词数量:", len(word))

#将tf-idf矩阵抽取出来，元素w[i][j]表示j词在i类文本中的tf-idf权重
X = coo_matrix(tfidf, dtype=np.float32).toarray() #稀疏矩阵 注意float

rst = X[0]
for i, x in enumerate(X):
    if i == 0:
        continue
    rst = rst + x
print(rst.shape)
word_tfidf={}
for i, tfidf_val in enumerate(rst):
    word_tfidf[i] = tfidf_val
# rst.sort()
# print(rst)
a = sorted(rst,reverse=1)
# print(a)
for key in word_tfidf:
    for i in range(20):
        if word_tfidf[key] == a[i]:
            print(word[key])

print(type(X))

单词数量: 2452
(2452,)
不了
不要
京东
什么
信号
充电
几天
刚买
垃圾
客服
就是
收到
有点
没有
耳机
质量
赠品
退货
问题
降价
<class 'numpy.ndarray'>
